# DistilBERT Fine-tuning vs Zero-Shot Baseline (Toy Intent Dataset)

This notebook loads a toy 15-intent dataset, evaluates a zero-shot baseline (`facebook/bart-large-mnli`), fine-tunes DistilBERT, and compares Accuracy/Macro-F1 on the test set.

In [8]:
# 🚀 Environment Setup Cell
# Run this once at the start of the notebook


import os
import subprocess

# Optional: create and activate a virtual environment
# (if you are running locally and want isolation)
venv_dir = "intent_env"

if not os.path.exists(venv_dir):
    subprocess.run([sys.executable, "-m", "venv", venv_dir])
    print(f"✅ Created virtual environment: {venv_dir}")
    # On Linux/macOS, to activate manually later: source intent_env/bin/activate
    # On Windows (PowerShell): .\\intent_env\\Scripts\\Activate.ps1

# Install all dependencies into the current environment
# 🚀 Environment Setup Cell (safe with pinned versions)
# Run this once at the start of the notebook

!pip install --upgrade pip
!pip install -q \
    torch==2.3.1 \
    transformers==4.44.2 \
    datasets==2.20.0 \
    evaluate==0.4.2 \
    accelerate==0.34.2 \
    scikit-learn==1.5.1 \
    pandas==2.2.2 \
    numpy==1.26.4 \
    fastapi \
    uvicorn


  Using cached pip-25.2-py3-none-any.whl (1.8 MB)


ERROR: To modify pip, please run the following command:
e:\PROJECTS_RK\Carnot\intent_env\Scripts\python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Setup

In [3]:

# !pip -q install -U transformers datasets evaluate scikit-learn accelerate
import os
os.environ["USE_TF"] = "0"

import random, numpy as np, pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, pipeline
from datasets import Dataset, DatasetDict

SEED = 42
random.seed(SEED); np.random.seed(SEED)

# Load dataset
CANDIDATES = ['/mnt/data/intent_dataset.csv','./intent_dataset.csv','/content/intent_dataset.csv']
data_path = next((p for p in CANDIDATES if os.path.exists(p)), None)
assert data_path, "intent_dataset.csv not found. Upload it first."
df = pd.read_csv(data_path)

labels = sorted(df['label'].unique().tolist())
label2id = {l:i for i,l in enumerate(labels)}
id2label = {i:l for l,i in label2id.items()}

train_df = df[df['split']=='train'].copy()
val_df   = df[df['split']=='val'].copy()
test_df  = df[df['split']=='test'].copy()

def to_ds(d): return Dataset.from_pandas(d[['text','label']].reset_index(drop=True))
raw = DatasetDict({'train': to_ds(train_df), 'validation': to_ds(val_df), 'test': to_ds(test_df)})
print(raw)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 15
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 15
    })
})


## Zero-shot Baseline (BART-MNLI)

In [4]:

zs = pipeline('zero-shot-classification', model='facebook/bart-large-mnli', device=-1)
candidate_labels = labels

def zs_predict(texts):
    pred = []
    for t in texts:
        r = zs(t, candidate_labels=candidate_labels, multi_label=False)
        pred.append(r['labels'][0])
    return pred

zs_preds = zs_predict(test_df['text'].tolist())
zs_acc = accuracy_score(test_df['label'], zs_preds)
zs_f1  = f1_score(test_df['label'], zs_preds, average='macro')
print(f"Zero-shot Accuracy: {zs_acc:.4f}\nZero-shot Macro-F1: {zs_f1:.4f}")
print("\nReport:\n", classification_report(test_df['label'], zs_preds, digits=3))


e:\PROJECTS_RK\Carnot\intent_env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Zero-shot Accuracy: 0.6000
Zero-shot Macro-F1: 0.5222

Report:
               precision    recall  f1-score   support

 book_flight      1.000     1.000     1.000         1
cancel_alarm      1.000     1.000     1.000         1
     goodbye      1.000     1.000     1.000         1
       greet      0.500     1.000     0.667         1
    navigate      0.500     1.000     0.667         1
 order_pizza      1.000     1.000     1.000         1
  play_music      0.000     0.000     0.000         1
    reminder      0.333     1.000     0.500         1
  send_email      0.000     0.000     0.000         1
   set_alarm      0.000     0.000     0.000         1
  small_talk      0.000     0.000     0.000         1
   tell_joke      0.000     0.000     0.000         1
   thank_you      0.000     0.000     0.000         1
        time      1.000     1.000     1.000         1
     weather      1.000     1.000     1.000         1

    accuracy                          0.600        15
   macro avg    

e:\PROJECTS_RK\Carnot\intent_env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\PROJECTS_RK\Carnot\intent_env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\PROJECTS_RK\Carnot\intent_env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Fine-tune DistilBERT

In [5]:
from transformers import EarlyStoppingCallback

model_name = 'distilbert-base-uncased'
tok = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

# tokenize and encode labels
def preprocess(example):
    enc = tok(example['text'], truncation=True)
    enc["labels"] = label2id[example['label']]   # numeric label
    return enc

tokd = raw.map(preprocess)

# 🚨 Drop the original string column 'label'
tokd = tokd.remove_columns(["label"])

collator = DataCollatorWithPadding(tokenizer=tok)

# metrics
def metrics(eval_pred):
    logits, y_true = eval_pred
    y_pred = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "macro_f1": f1_score(y_true, y_pred, average='macro')
    }

args = TrainingArguments(
    output_dir='./outputs',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    num_train_epochs=25,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='macro_f1',
    logging_steps=5,
    seed=SEED,
    report_to=[],
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokd['train'],
    eval_dataset=tokd['validation'],
    tokenizer=tok,
    data_collator=collator,
    compute_metrics=metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]

)

# train
trainer.train()

trainer.save_model("distilbert")  # For DistilBERT


# evaluate on test
test_metrics = trainer.evaluate(tokd['test'])
print("Test metrics:", test_metrics)

# predictions
pred_output = trainer.predict(tokd['test'])
preds = np.argmax(pred_output.predictions, axis=1)
true  = pred_output.label_ids

# classification report
print("\nClassification report (fine-tuned):\n",
      classification_report(true, preds,
                            labels=list(label2id.values()),
                            target_names=labels,
                            digits=3,
                            zero_division=0))


e:\PROJECTS_RK\Carnot\intent_env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 15/15 [00:00<00:00, 1301.85 examples/s]
e:\PROJECTS_RK\Carnot\intent_env\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Trans

{'loss': 2.7227, 'grad_norm': 2.9186813831329346, 'learning_rate': 4.8333333333333334e-05, 'epoch': 0.83}


                                               
  4%|▍         | 6/150 [00:10<03:08,  1.31s/it]

{'eval_loss': 2.673696279525757, 'eval_accuracy': 0.06666666666666667, 'eval_macro_f1': 0.01111111111111111, 'eval_runtime': 0.2285, 'eval_samples_per_second': 65.639, 'eval_steps_per_second': 8.752, 'epoch': 1.0}


  7%|▋         | 10/150 [00:17<03:08,  1.34s/it]

...

{'train_runtime': 196.3655, 'train_samples_per_second': 5.729, 'train_steps_per_second': 0.764, 'train_loss': 0.9713185540835063, 'epoch': 25.0}


100%|██████████| 2/2 [00:00<00:00, 22.35it/s]


Test metrics: {'eval_loss': 0.9929536581039429, 'eval_accuracy': 0.8, 'eval_macro_f1': 0.7777777777777778, 'eval_runtime': 0.1788, 'eval_samples_per_second': 83.89, 'eval_steps_per_second': 11.185, 'epoch': 25.0}


100%|██████████| 2/2 [00:00<00:00, 19.31it/s]



Classification report (fine-tuned):
               precision    recall  f1-score   support

 book_flight      1.000     1.000     1.000         1
cancel_alarm      1.000     1.000     1.000         1
     goodbye      0.000     0.000     0.000         1
       greet      0.000     0.000     0.000         1
    navigate      1.000     1.000     1.000         1
 order_pizza      1.000     1.000     1.000         1
  play_music      1.000     1.000     1.000         1
    reminder      0.500     1.000     0.667         1
  send_email      1.000     1.000     1.000         1
   set_alarm      0.000     0.000     0.000         1
  small_talk      1.000     1.000     1.000         1
   tell_joke      1.000     1.000     1.000         1
   thank_you      1.000     1.000     1.000         1
        time      1.000     1.000     1.000         1
     weather      1.000     1.000     1.000         1

    accuracy                          0.800        15
   macro avg      0.767     0.800     0.77

## RoBERTa-base

In [6]:
import os, random, numpy as np, pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, DataCollatorWithPadding,
    EarlyStoppingCallback
)
from datasets import Dataset, DatasetDict
model_name = "roberta-base"
tok = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

# tokenize and encode labels
def preprocess(example):
    enc = tok(example['text'], truncation=True)
    enc["labels"] = label2id[example['label']]   # numeric label
    return enc

tokd = raw.map(preprocess)

# 🚨 Drop the original string column 'label'
tokd = tokd.remove_columns(["label"])

collator = DataCollatorWithPadding(tokenizer=tok)


# ------------------------
# Metrics
# ------------------------
def metrics(eval_pred):
    logits, y_true = eval_pred
    y_pred = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "macro_f1": f1_score(y_true, y_pred, average='macro')
    }

# ------------------------
# Training Arguments
# ------------------------
args = TrainingArguments(
    output_dir='./outputs_roberta',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,  # smaller for RoBERTa
    num_train_epochs=20, # more epochs, with early stopping
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='macro_f1',
    logging_steps=10,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    seed=SEED,
    report_to=[],
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokd['train'],
    eval_dataset=tokd['validation'],
    tokenizer=tok,
    data_collator=collator,
    compute_metrics=metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# ------------------------
# Train
# ------------------------
trainer.train()

trainer.save_model("roberta")


# ------------------------
# Evaluate on Test
# ------------------------
test_metrics_roberta = trainer.evaluate(tokd['test'])
print("\nTest metrics:", test_metrics_roberta)

# Predictions + Report
preds = trainer.predict(tokd['test']).predictions.argmax(axis=1)
true  = [label2id[l] for l in test_df['label']]

print("\nClassification Report (RoBERTa-base):\n",
      classification_report(true, preds, target_names=labels, digits=3))

e:\PROJECTS_RK\Carnot\intent_env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 15/15 [00:00<00:00, 1380.49 examples/s]
e:\PROJECTS_RK\Carnot\intent_env\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of

{'eval_loss': 2.7162394523620605, 'eval_accuracy': 0.06666666666666667, 'eval_macro_f1': 0.008333333333333333, 'eval_runtime': 0.2541, 'eval_samples_per_second': 59.032, 'eval_steps_per_second': 7.871, 'epoch': 1.0}


  8%|▊         | 10/120 [00:22<03:18,  1.80s/it]

{'loss': 2.7246, 'grad_norm': 2.4551167488098145, 'learning_rate': 5e-06, 'epoch': 1.67}


 10%|█         | 12/120 [00:25<02:53,  1.61s/it]

...

100%|██████████| 2/2 [00:00<00:00, 18.17it/s]



Test metrics: {'eval_loss': 1.124314785003662, 'eval_accuracy': 0.7333333333333333, 'eval_macro_f1': 0.6888888888888889, 'eval_runtime': 0.238, 'eval_samples_per_second': 63.033, 'eval_steps_per_second': 8.404, 'epoch': 20.0}


100%|██████████| 2/2 [00:00<00:00, 16.74it/s]


Classification Report (RoBERTa-base):
               precision    recall  f1-score   support

 book_flight      1.000     1.000     1.000         1
cancel_alarm      0.000     0.000     0.000         1
     goodbye      0.000     0.000     0.000         1
       greet      0.000     0.000     0.000         1
    navigate      1.000     1.000     1.000         1
 order_pizza      1.000     1.000     1.000         1
  play_music      0.000     0.000     0.000         1
    reminder      1.000     1.000     1.000         1
  send_email      1.000     1.000     1.000         1
   set_alarm      0.500     1.000     0.667         1
  small_talk      1.000     1.000     1.000         1
   tell_joke      1.000     1.000     1.000         1
   thank_you      1.000     1.000     1.000         1
        time      1.000     1.000     1.000         1
     weather      0.500     1.000     0.667         1

    accuracy                          0.733        15
   macro avg      0.667     0.733     0.


e:\PROJECTS_RK\Carnot\intent_env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\PROJECTS_RK\Carnot\intent_env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\PROJECTS_RK\Carnot\intent_env\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
import pandas as pd

# ------------------------
# Collect Results
# ------------------------
results = []

# 1. Zero-shot (BART-MNLI)
results.append({
    "model": "Zero-shot (BART-MNLI)",
    "accuracy": zs_acc,
    "macro_f1": zs_f1
})

# 2. DistilBERT fine-tuned
results.append({
    "model": "DistilBERT (fine-tuned)",
    "accuracy": test_metrics['eval_accuracy'],   # from distilbert trainer.evaluate()
    "macro_f1": test_metrics['eval_macro_f1']
})

# 3. RoBERTa-base fine-tuned
results.append({
    "model": "RoBERTa-base (fine-tuned)",
    "accuracy": test_metrics_roberta['eval_accuracy'],   # from roberta trainer.evaluate()
    "macro_f1": test_metrics_roberta['eval_macro_f1']
})

# ------------------------
# Save Results to CSV
# ------------------------
df_results = pd.DataFrame(results)
df_results.to_csv("results.csv", index=False)

print("\n✅ Results saved to results.csv:\n")
print(df_results)



✅ Results saved to results.csv:

                       model  accuracy  macro_f1
0      Zero-shot (BART-MNLI)  0.600000  0.522222
1    DistilBERT (fine-tuned)  0.800000  0.777778
2  RoBERTa-base (fine-tuned)  0.733333  0.688889


## Notes
- Tiny dataset ⇒ variance across runs.
- Zero-shot depends on label phrasing; try natural-language label descriptions.
- Replace `intent_dataset.csv` with your dataset.